In [1]:
import glob
from modules.ontology_classes import CogafInstance
from modules.emotion import getEmotion

In [2]:
txt_files = glob.glob('./../data/text/prep/*.txt')
print(len(txt_files))
print(txt_files[0])

text_list = []
for file in txt_files:
    with open(file, 'r', encoding='utf-8') as f:
        text_list.append(f.read())

84
./../data/text/prep\0000_000_999-accidentes-trabajo.txt


In [3]:
response = getEmotion(text_list[1])

In [4]:
response.status_code

414

In [5]:
response.content

b'<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\n<html><head>\n<title>414 Request-URI Too Long</title>\n</head><body>\n<h1>Request-URI Too Long</h1>\n<p>The requested URL\'s length exceeds the capacity\nlimit for this server.<br />\n</p>\n</body></html>\n'

In [12]:
response = getEmotion("""
Aprendidas

Auxiliar de tráfico (Paletero - Señalelo)
 aprisionado por un vehículo

Tránsito

¿Qué paso?

El trabajador se encontraba coordinando el flujo vehicular en una obra y al momento de dar vía uno de los conductores no lo observa y es tropellado por este causándole la muerte instantánea al aprisionándolo contra otro vehículo.

¿Qué lo causó?

Fallas en los protocolos o estándares Falencias en el proceso de visualización
 de seguridad en la actividad. tanto de las señales como del señalero.

Ubicación inadecuada del trabajador. Fallas en la comunicación entre conductores y señalero.
""")

In [13]:
response.status_code

200

In [14]:
response.content

b'grief (75.34%) & loathing (3.27%) [INTROSPECTION=-90.86%,TEMPER=-53.6%,ATTITUDE=-67.08%,SENSITIVITY=-64.7%]\n'